<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python -m pip install --upgrade pip
!pip install pyLDAvis
# import pyLDAvis
import pyLDAvis.gensim_models # don't skip this

from pprint import pprint
import time
import pandas as pd
import re
import string
import nltk
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import collections

try:
    collectionsAbc = collections.abc
except AttributeError:
    collectionsAbc = collections 
try:
    from collections.abc import Callable  # noqa
except ImportError:
    from collections import Callable  # noqa


from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import MiniBatchKMeans

In [3]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

In [ ]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

bug_report = pd.read_csv(csv_file)
bug_report.head(5)

In [122]:
%%time

bug_report = bug_report.drop(columns=['Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
bug_report = bug_report.dropna(axis=0, subset=['Description'])
bug_report[bug_report['Description'].map(len) > 4]

bug_report.head(1)

CPU times: user 72.2 ms, sys: 221 µs, total: 72.4 ms
Wall time: 73.3 ms


,Issue_id,Priority,Duplicated_issue,Title,Description
0,1,P3,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...


In [123]:
%%time

bug_report["Description"] = bug_report["Description"].str.replace("Fixed in HEAD", "")
bug_report["Description"] = bug_report["Description"].str.replace("Has been marked as read-only", " ")

CPU times: user 348 ms, sys: 2.29 ms, total: 351 ms
Wall time: 353 ms


In [124]:
%%time

def cleanTextFromDescriptionAndTitle(report):

  report = report.replace('\t', '')
  report = re.sub('\w*\d\w*', '', report)
  report = report.lower()
  report = report.translate(str.maketrans('', '', string.punctuation))
    
  return report

bug_report['Description'] = bug_report['Description'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))
bug_report['Title'] = bug_report['Title'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))

bug_report.loc[[0]]

CPU times: user 10.4 s, sys: 0 ns, total: 10.4 s
Wall time: 10.8 s


,Issue_id,Priority,Duplicated_issue,Title,Description
0,1,P3,NaN,usability issue with external editors,setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with an external editor found the modified file on disk and showed the changes the real problem occurs if release actually finds something to release but you dont spot that some resources are missing this is extremely error prone one of my changes didnt made it into build because of this notes eg pm release should do a refresh from local before doing the release moving to vcm km pm comments from jm in related email should not do this for free could have a setting which made it optoinal but...


In [ ]:
train = bug_report
train.head()

In [ ]:
%%time

def wordTokenizationAndLemmatization(report):

    new_report = []
    for token in SIMPLE_PREPROCESS(report):
        if token not in STOP_WORDS:
          if len(token) > 5:
            new_report.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return new_report

bug_report['Title'] = bug_report['Title'].apply(lambda x: wordTokenizationAndLemmatization(x))
bug_report['Description'] = bug_report['Description'].apply(lambda x: wordTokenizationAndLemmatization(x))

bug_report.head()

In [ ]:
%%time

duplicated_reports = bug_report.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
duplicated_reports.head(1)

In [ ]:
%%time

master_reports = bug_report[bug_report.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
master_reports.head(5)

In [ ]:
%%time

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for key, value in dictionary.iteritems():
  print(key, value)
  count += 1
  if count > 100:
      break


In [ ]:
%%time

bag_of_words_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in master_reports['Description']]
bag_of_words_corpus[0]

In [ ]:
%%time

bag_of_words_doc = bag_of_words_corpus[0]

for i in range(len(bag_of_words_doc)):
    print("{}. \"{}\" word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

In [ ]:
%%time

corpus = bag_of_words_corpus

lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

pprint(lda_model.print_topics())

In [14]:
file = open('drive/My Drive/dictionary.pickle', 'wb')

# dump information to that file
pickle.dump(dictionary, file)

In [15]:
f= open('drive/My Drive/bow_corpus.pickle', 'wb')

# dump information to that file
pickle.dump(bag_of_words_corpus, f)

In [ ]:
lda_model.save('drive/My Drive/lda_model.model')

In [ ]:
%%time

print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=bug_report['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
%%time

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

In [ ]:
unseen_document = "setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with."


bow_vector = topic=lda_model[dictionary.doc2bow(wordTokenizationAndLemmatization(unseen_document))]

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))

Score: 0.5446290969848633	 Topic: 0.018*"target" + 0.017*"extension" + 0.015*"support" + 0.014*"change" + 0.013*"property" + 0.012*"provide" + 0.011*"content" + 0.011*"define" + 0.010*"default" + 0.010*"currently"
Score: 0.050613515079021454	 Topic: 0.093*"project" + 0.031*"create" + 0.027*"workspace" + 0.021*"change" + 0.018*"package" + 0.016*"delete" + 0.016*"eclipse" + 0.016*"folder" + 0.015*"resource" + 0.014*"explorer"
Score: 0.05060430243611336	 Topic: 0.050*"select" + 0.035*"dialog" + 0.028*"search" + 0.028*"action" + 0.025*"button" + 0.022*"context" + 0.019*"selection" + 0.017*"change" + 0.015*"compare" + 0.014*"filter"
Score: 0.05060174688696861	 Topic: 0.060*"editor" + 0.032*"window" + 0.024*"perspective" + 0.022*"eclipse" + 0.018*"daemon" + 0.014*"switch" + 0.013*"workbench" + 0.013*"editors" + 0.013*"scroll" + 0.013*"open"
Score: 0.050595398992300034	 Topic: 0.029*"create" + 0.027*"reproduce" + 0.024*"windows" + 0.021*"attachment" + 0.019*"attach" + 0.018*"problem" + 0.017*

In [ ]:
%%time


for c in range(10):
    exec('topic_{} = pd.DataFrame()'.format(c))
    for i in range(len(master_reports)):
        topic=lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(c,c))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(c,c))


In [16]:
%%time

for c in range(10):
  exec('topic_{} = pd.read_csv("topic_{}.csv", error_bad_lines=False, engine="python")'.format(c,c))
  exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
  exec("topic_{}['Description'] = topic_{}['Description'].map(wordTokenizationAndLemmatization)".format(c,c))

CPU times: user 11.2 s, sys: 208 ms, total: 11.4 s
Wall time: 13 s


In [17]:
%%time

for i in range(10):
  exec('sent_{} = []'.format(i))
  exec('x = topic_{}'.format(i))
  for j in range(len(x)):
    exec('sent_{}.append(topic_{}.Description[{}])'.format(i,i,j))

CPU times: user 1.14 s, sys: 3.92 ms, total: 1.14 s
Wall time: 1.15 s


In [ ]:
%%time

#Length of all the cluster
for sent in range(10):
  exec('print(len(sent_{}))'.format(sent))
  

In [ ]:
!pip install glove-python-binary

In [20]:
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile
from glove import Glove, Corpus
import glove

In [ ]:
%%time

for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent_{}'.format(cluster))    # Path to a corpus file
  vector_size = 100                           # Dimensionality of the word vectors
  w = 6                                       #Maximum distance between the current and predicted word within a sentence
  min_count = 5                               # Ignores all words with total frequency lower than this.
  CBoW = 0                                    # Training algorithm: 1 for skip-gram; otherwise CBOW(The model tries to predict the target word by trying to understand the context of the surrounding words.).
  iterations = 100

  exec('w2vmodel{} = Word2Vec(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  exec('path = get_tmpfile("word2vec{}.model")'.format(cluster))
  exec('w2vmodel{}.save("word2vec{}.model")'.format(cluster, cluster))

In [ ]:
%%time

for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent_{}'.format(cluster))
  vector_size = 100
  w = 6
  min_count = 5
  CBoW = 0
  iterations = 100

  #Training FastText model for each cluster
  exec ('ftmodel{} = FastText(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("ftmodel{}.model")'.format(cluster))
  exec('ftmodel{}.save("ftmodel{}.model")'.format(cluster, cluster))

In [ ]:
%%time

for cluster in range(10):
  vector_size = 100
  exec('glove_corpus{}=Corpus()'.format(cluster, cluster)) 
  exec('glove_corpus{}.fit(sent_{})'.format(cluster, cluster))
  exec('glove{}= Glove(no_components=vector_size, learning_rate=0.18, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)'.format(cluster, cluster))
  exec('glove{}.fit(glove_corpus{}.matrix, epochs=1000, no_threads=3, verbose=True)'.format(cluster, cluster))
  exec('transformer = lambda dictionary2:glove{}.transform_paragraph(words, epochs=1000,ignore_missing=False)'.format(cluster, cluster))
  exec('glove{}.add_dictionary(glove_corpus{}.dictionary)'.format(cluster, cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("glove{}.model")'.format(cluster))
  exec('glove{}.save("glove{}.model")'.format(cluster, cluster))

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from scipy.spatial import distance
warnings.filterwarnings("ignore", category=DeprecationWarning)
from glove import Glove, Corpus

In [22]:
f = open('drive/My Drive/bow_corpus.pickle', 'rb')
bow_corpus=pickle.load(f)

file = open('drive/My Drive/dictionary.pickle', 'rb')
dictionary=pickle.load(file)

# later on, load trained model from file
lda_model =  models.LdaModel.load('drive/My Drive/lda_model.model')

In [66]:
test = duplicated_reports
test.head()
print(len(test))

14398


In [24]:
for mod in range(10):
  #import all the trained Word2Vec models
  exec('w2vmodel{} = Word2Vec.load("word2vec{}.model")'.format(mod, mod))

  #import all the trained FastText models
  exec('ftmodel{} = FastText.load("ftmodel{}.model")'.format(mod, mod))

  #import all the trained GloVe models
  exec('glove{} = Glove.load("glove{}.model")'.format(mod, mod))

In [58]:
def sim_with_clusters_lda_topn(DR, n):
    vec_bow = dictionary.doc2bow(DR)
    x= lda_model[vec_bow]
    topic = np.asarray(x)
    # max_sim = int(topic[np.argmax(topic[:,1]),0]) 
    # max_sim
    sim=[]
    x= topic[np.argsort(topic[:,1])[-n:][::-1],0]
    for i in range(len(x)):
        sim.append(int(x[i]))
    # return max_sim
    return sim


In [26]:
def average_word_vectors_glove(words, model, vocabulary, num_features):  
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.  

    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.word_vectors[model.dictionary[word]])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    


def averaged_word_vectorizer_glove(corpus, model, num_features):
    vocabulary = set(model.dictionary)
    if(any(isinstance(i, list) for i in corpus)):
      features = [average_word_vectors_glove(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
      return np.array(features)
    else:
      features = average_word_vectors_glove(corpus, model, vocabulary, num_features)
      return np.array(features)


In [27]:
def average_word_vectors_w2v(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    
def averaged_word_vectorizer_w2v(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    if(any(isinstance(i, list) for i in corpus)):
      features = [average_word_vectors_w2v(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
      return np.array(features)
    else:
      features = average_word_vectors_w2v(corpus, model, vocabulary, num_features)
      return np.array(features)

In [28]:
def sim(vec1, vec2): 
  sim1 = 1/(1+distance.euclidean(vec1, vec2))
  sim2 = cosine_similarity(vec1, vec2)
  sim=(sim1+sim2)/2 
  return sim

In [29]:
def fusion(vec1, vec2, vec3, vec4, fusion_no):
    if (fusion_no == '4'):
        vec3 = [vec3]
        vec4 = [vec4]
        avg1 = (np.add(vec1, vec2))/2
        pca = PCA(n_components=100)
        avg_fit = pca.fit(avg1)
        master = pca.transform(avg1)
        avg2 = (np.add(vec3, vec4))/2
        vec_duplicate = pca.transform(avg2)
        return vec_duplicate, master

    else:
        raise ValueError('Invalid value for fusion')

In [30]:
# creation of feature vectors by multimodality feature extraction
def feature_vectors_multi_modality(DR, corpus, model1, model2, fusion_no):
  master_ft1 = averaged_word_vectorizer_w2v(corpus=sent, model=model1, num_features=100)
  master_glove2 = averaged_word_vectorizer_glove(corpus=sent, model=model2, num_features=100)

  vec_duplicate1 = averaged_word_vectorizer_w2v(corpus=DR, model=model1, num_features=100)
  vec_duplicate2 = averaged_word_vectorizer_glove(corpus=DR, model=model2, num_features=100)

  vec_duplicate , master= fusion(master_ft1, master_glove2, vec_duplicate1, vec_duplicate2, fusion_no)

  return vec_duplicate,master

In [116]:
def compare_topn(model1, model2, cluster, sent, DR, topn, modal, fusion_no):
  similarity=[]

  if (modal == 'multi'):
  #create feature vectors for duplicate and master reports using multimodality
    vec_duplicate, master= feature_vectors_multi_modality(DR, sent, model1, model2, fusion_no)
  else:
    raise ValueError('Invalid Modality entered')

  for doc in range(len(master)):
    vec_master = master[doc]
    vec_master= [vec_master]
    unified_sim = sim(vec_duplicate, vec_master)

    similarity.append(unified_sim)

  similarity = np.asarray(similarity)
  similarity= np.concatenate(similarity, axis=0 )
  similarity= np.concatenate(similarity, axis=0 )
  max_similar_reports=similarity.argsort()[topn:][::-1]

  return(max_similar_reports)

In [130]:
# Recall Rate for Top-2.5K reports (Because Top-N where N = n * topn (2.5K = 3*833)) 

import random
topic = random.randint(0, 14398)

sample = test.Description[topic] #The test sample (duplicate report)
max_cluster = sim_with_clusters_lda_topn(sample, 1)
print(max_cluster)

for max in max_cluster:
    exec('cluster = topic_{}'.format(max))              #The predicted cluster
    exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
    exec('model2 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
    exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
    topn = 10                                          #The number of predicted master report for single predicted cluster
    fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
    modal = 'multi'                                    # Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)

    max_sim = compare_topn(model1, model2, cluster, sent, sample, topn, modal, fusion_no)  #This will return the Top-N predicted master reports
    print(max_sim)
    print(type(max_sim))
   


[9]
[1318  914 1666 ... 2710 1645 1247]
<class 'numpy.ndarray'>
